In [12]:
import os
import json
import numpy as np

In [17]:
def load_data(data_path, split):
    # load dataset from the disk
    with open(os.path.join(data_path, "metadata.json")) as f:
        metadata = json.load(f)
    with open(os.path.join(data_path, f"{split}_offset.json")) as f:
        offset = json.load(f)
    offset = {int(k): v for k, v in offset.items()}

    particle_type = np.memmap(os.path.join(data_path, f"{split}_particle_type.dat"), dtype=np.int64, mode="r")
    position = np.memmap(os.path.join(data_path, f"{split}_position.dat"), dtype=np.float32, mode="r")

    return metadata, offset, particle_type, position

In [18]:
metadata, offset, particle_type, position = load_data("../datasets/WaterDropSmall/", "test")

In [22]:
metadata

{'bounds': [[0.1, 0.9], [0.1, 0.9]],
 'sequence_length': 1000,
 'default_connectivity_radius': 0.015,
 'dim': 2,
 'dt': 0.0025,
 'vel_mean': [-3.964619574176163e-05, -0.00026272129664401046],
 'vel_std': [0.0013722809722366911, 0.0013119977252142715],
 'acc_mean': [2.602686518497945e-08, 1.0721623948191945e-07],
 'acc_std': [6.742962470925277e-05, 8.700719180424815e-05]}

In [21]:
offset

{0: {'particle_type': {'offset': 0, 'shape': [295]},
  'position': {'offset': 0, 'shape': [1001, 295, 2]}},
 1: {'particle_type': {'offset': 295, 'shape': [803]},
  'position': {'offset': 590590, 'shape': [1001, 803, 2]}},
 2: {'particle_type': {'offset': 1098, 'shape': [410]},
  'position': {'offset': 2198196, 'shape': [1001, 410, 2]}},
 3: {'particle_type': {'offset': 1508, 'shape': [997]},
  'position': {'offset': 3019016, 'shape': [1001, 997, 2]}},
 4: {'particle_type': {'offset': 2505, 'shape': [271]},
  'position': {'offset': 5015010, 'shape': [1001, 271, 2]}}}

In [20]:
particle_type.shape, position.shape

((2776,), (5557552,))